# Integer minimum cost flows with separable convex cost objective

The Problem of finding the most likely mulitpath split for a payment pair on the lightning network can be reduced to an **integer minimum cost flows with separable convex cost objective** and yields a polynomial solver c.f.: https://twitter.com/renepickhardt/status/1385144337907044352

This notebook tries to implement a solution given in the Textbook Network Flows Network Flows: Theory, Algorithms, and Applications by Ravindra K. Ahuja, Thomas L. Magnanti and James B. Orlin

This approach follows mainly chapter 9, 10.2 and 14.5 of the Textbook. 

Other good resources are the Lecture series http://courses.csail.mit.edu/6.854/20/ by David Karger (http://people.csail.mit.edu/karger/) with lecture notes at http://courses.csail.mit.edu/6.854/current/Notes/ and more specificially http://courses.csail.mit.edu/6.854/current/Notes/n09-mincostflow.html and the relevant subset of the recorded videos at this playlist: https://www.youtube.com/playlist?list=PLaRKlIqjjguDXlnJWG2T7U52iHZl8Edrcv

Other good resources are the lecture nodes by Hochbaum: https://hochbaum.ieor.berkeley.edu/

I did not find an open source implementation. Code for similar problems seems to be part of this library: https://github.com/frangio68/Min-Cost-Flow-Class but it seems to only allow quadratic cost functions. 

Following previous research ( https://arxiv.org/abs/2103.08576 ) our cost function is f(x) = log(c) - log(c-x) which turns out to be convex (f'(x) = 1/(c-x) and f"(x) = 1/(c-x)^2 from which we can see that f"(x) > 0)

In [3]:
import json
import networkx as nx
import math

log = math.log2

## some helpers for graphs

introduce some helper functions for handling graphs and importing out data structure / example

In [146]:
def import_channel_graph():
    # retrieve this by: lightning-cli listchannels > listchannels.json
    f = open("listchannels.json")
    jsn = json.load(f)
    G = nx.Graph()
    for channel in jsn["channels"]:
        src = channel["source"]
        dest = channel["destination"]
        cap = int(int(channel["satoshis"])/1000)
        sid = channel["short_channel_id"]
        G.add_edge(src,dest,cap=cap,sid=sid)
    return G

channel_graph = import_channel_graph()

G = nx.DiGraph()
G.add_edge("S","A",cap=2)
G.add_edge("S","X",cap=1)
G.add_edge("A","B",cap=2)
G.add_edge("X","B",cap=9)
G.add_edge("X","Y",cap=7)
G.add_edge("Y","D",cap=4)
G.add_edge("B","D",cap=4)


def next_hop(path):
    for i in range(1,len(path)):
        src = path[i-1]
        dest = path[i]
        yield (src,dest)
        
def path_str(path):
    it = path[0]
    if len(it)>5:
        it = it[:5]
    res = it
    for src, dest in next_hop(path):
        if len(dest)>5:
            dest = dest[:5]
        res += " {}".format(dest)
    return res

def print_residual(r,G,x):
    for i,j in G.edges():
        cap = G[i][j]["cap"]
        f = x[i][j]
        #print("src: {} dest: {} cap: {} flow: {}".format(i,j,cap,f))
        if r.has_edge(i,j):
            print(i,j,r[i][j])
        if r.has_edge(j,i):
            print(j,i,r[j][i])   

SRC = "03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df"
DEST = "022c699df736064b51a33017abfc4d577d133f7124ac117d3d9f9633b6297a3b6a"
FLOW = 9200

In [147]:
def generate_null_flow(G):
    flow = {n:{} for n in G.nodes()}
    for u,v in G.edges():
        flow[u][v]=0
        flow[v][u]=0
    return flow

def cost(x,c):
    """if c%2==0:
        #print(x)
        return 4* x**2
    else:
        return x**2"""

    return log(c+1)-log(c+1-x)    




## Compute Detla residual network as a unit capacity network

this means that if an edge is in the residual network we can flow delta in the actual network and otherwise we can't

as the delta residual network only cares about reduced cost the fuction also gets the potentials passed

In [155]:
def compute_delta_residual_network(G,x,delta,pi,C=cost):
    residual = nx.MultiDiGraph()
    for i, j in G.edges():
        cap = G[i][j]["cap"]
        f = x[i][j]
        if f+delta <=cap:
            unit_cost = (C(f + delta, cap) - C(f,cap))/delta
            residual.add_edge(i,j,uc= unit_cost,rc = unit_cost - pi[i] + pi[j],flow = 0,capacity=1,orgiginal_cap=cap,key=0,k=0)
        if delta <= f:#backward flow
            unit_cost = (C(f - delta, cap) - C(f,cap))/(delta)
            residual.add_edge(j,i,uc= unit_cost,rc=unit_cost - pi[j] + pi[i], flow = 0,capacity=1,orgiginal_cap=cap,key=1,k=1)
        
        f = x[j][i]
        if f+delta <=cap:
            unit_cost = (C(f + delta, cap) - C(f,cap))/delta
            residual.add_edge(j,i,uc= unit_cost,rc=unit_cost - pi[j] + pi[i],flow = 0,capacity=1,orgiginal_cap=cap,key=0,k=0)
        if delta <= f:#backward flow
            unit_cost = (C(f - delta, cap) - C(f,cap))/(delta)
            residual.add_edge(i,j,uc= unit_cost,rc=unit_cost - pi[i]+ pi[j],flow = 0, capacity=1,orgiginal_cap=cap,key=1,k=1) 
    return residual

In [188]:
def fulfill_reduced_cost_optimiality_rule(R,x,e,delta):
    to_remove=[]
    for i,j, data in R.edges(data=True):
        if data["rc"] < 0:
            #print("found non optimal edege: ",i,j,data["rc"],data)
            cap = data["orgiginal_cap"]
            if x[j][i]>=delta:
                x[j][i]-=delta
                flag = True
                
            elif x[i][j]+delta <= cap:
                x[i][j]+=delta
                flag = True
            
            #if x[i][j]>0:
            #    print("!!!! ther was never flow in the direction in which the residual edges are not cost optimal",i,j,x[i][j])
            
            
            #TODO: test what happens if residual is recomputed
            if flag:                
                #to_remove.append((i,j,data["k"]))
                for k,v in R[i][j].items():
                    if (v["k"]==data["k"]):
                        R[i][j][k]["flow"]=1
                    #R[i][j][]
                
                e[i]-=delta
                e[j]+=delta
        #else:
        #    print("optimal", i,j,data["rc"], data)
            
#    for i,j,k in to_remove:
#        R.remove_edge(i,j,key=k)


In [194]:
def augment_path(flow,path,amt,R):
    for src,dest in next_hop(path):
        if flow[dest][src]>0:
            flow[dest][src]-=amt
            #if flow
        else:
            flow[src][dest]+=amt
        data = R[src][dest]
        
        if len(data)>1:
            if data[0]["flow"] ==0 and data[1]["flow"]==0:
                if data[0]["rc"]<data[1]["rc"]:
                    R[src][data][0]["flow"]=1
                else:
                    R[src][data][1]["flow"]=1
            else: 
                if data[0]["flow"]==0:
                    R[src][dest][0]["flow"]=1
                if data[1]["flow"]==0:
                    R[src][dest][1]["flow"]=1
        else: 
            R[src][dest][0]["flow"]=1
        #print(data)


In [205]:
def get_unsaturated_residual_subgraph(R):
    res = nx.MultiDiGraph()
    
    for i,j,data in R.edges(data=True):
        if data["flow"]!=data["capacity"]:
            if data["rc"]<0:
                print(i,j,data)
                print(R[i][j])
                print(R[j][i])
                raise Exception("asdf")
            res.add_edge(i,j,**data)
    
    return res

def recompute_reduced_cost(R,pi):
    res = nx.MultiDiGraph()
    for i,j,data in R.edges(data=True):
        data["rc"]=data["uc"]-pi[i]+pi[j]
        data["key"]=data["k"]
        res.add_edge(i,j,**data)
    return res


In [206]:

x = generate_null_flow(G)
pi = {n:0 for n in G.nodes}
R = compute_delta_residual_network(G,x,2,pi)
print_residual(R,G,x)

x["S"]["A"]=2
x["A"]["B"]=2
x["B"]["D"]=2

"""
pi["S"]-=0
pi["A"]-=R["S"]["A"][0]["rc"]
pi["B"]-=R["S"]["A"][0]["rc"]+R["A"]["B"][0]["rc"]
pi["X"]-=R["S"]["A"][0]["rc"]+R["A"]["B"][0]["rc"] + R["B"]["X"][0]["rc"]
pi["Y"]-=R["S"]["A"][0]["rc"]+R["A"]["B"][0]["rc"] + R["B"]["X"][0]["rc"] + R["X"]["Y"][0]["rc"]
pi["D"]-=R["S"]["A"][0]["rc"]+R["A"]["B"][0]["rc"] + R["B"]["D"][0]["rc"]
"""
print(paths["D"])
distances, paths = nx.single_source_dijkstra(R,"S",weight="rc")
            #distances, paths = nx.single_source_bellman_ford(R,k,weight="rc")
            #####print("Distances from {}".format(k))
for n, d_value in distances.items():
                #####print(n,d_value)
                #if n in pi:
    pi[n]-=d_value

#add hoc because known for our data set
e = {n:0 for n in G.nodes}
print(pi)

print(x)
R = compute_delta_residual_network(G,x,1,pi)
print("\n1-residual network after 2 flow S A B D")
print_residual(R,G,x)

fulfill_reduced_cost_optimiality_rule(R,x,e,1)
print_residual(R,G,x)
r = get_unsaturated_residual_subgraph(R)
print_residual(r,G,x)
print(e)
print(x)

distances, paths = nx.single_source_dijkstra(r,"X",weight="rc")
for n, d_value in distances.items():
                #####print(n,d_value)
                #if n in pi:
    pi[n]-=d_value
path = paths["D"]

R = recompute_reduced_cost(R,pi)

augment_path(x,path,1,R)
r = get_unsaturated_residual_subgraph(R)
print_residual(r,G,x)

distances, paths = nx.single_source_dijkstra(r,"Y",weight="rc")
for n, d_value in distances.items():
                #####print(n,d_value)
                #if n in pi:
    pi[n]-=d_value

path = paths["B"]
augment_path(x,path,1,R)
print(x)

S A {0: {'uc': 0.792481250360578, 'rc': 0.792481250360578, 'flow': 0, 'capacity': 1, 'orgiginal_cap': 2, 'k': 0}}
A S {0: {'uc': 0.792481250360578, 'rc': 0.792481250360578, 'flow': 0, 'capacity': 1, 'orgiginal_cap': 2, 'k': 0}}
A B {0: {'uc': 0.792481250360578, 'rc': 0.792481250360578, 'flow': 0, 'capacity': 1, 'orgiginal_cap': 2, 'k': 0}}
B A {0: {'uc': 0.792481250360578, 'rc': 0.792481250360578, 'flow': 0, 'capacity': 1, 'orgiginal_cap': 2, 'k': 0}}
X B {0: {'uc': 0.1609640474436811, 'rc': 0.1609640474436811, 'flow': 0, 'capacity': 1, 'orgiginal_cap': 9, 'k': 0}}
B X {0: {'uc': 0.1609640474436811, 'rc': 0.1609640474436811, 'flow': 0, 'capacity': 1, 'orgiginal_cap': 9, 'k': 0}}
X Y {0: {'uc': 0.20751874963942196, 'rc': 0.20751874963942196, 'flow': 0, 'capacity': 1, 'orgiginal_cap': 7, 'k': 0}}
Y X {0: {'uc': 0.20751874963942196, 'rc': 0.20751874963942196, 'flow': 0, 'capacity': 1, 'orgiginal_cap': 7, 'k': 0}}
B D {0: {'uc': 0.36848279708310305, 'rc': 0.36848279708310305, 'flow': 0, 'c

In [211]:
def capacity_scaling_min_cost_flow(s,d,U,G):
    x = generate_null_flow(G)
    pi = {n:0 for n in G.nodes}
    e = {n:0 for n in G.nodes}
    e[s]=U
    e[d]=-U
    delta = 2**int(log(U))
    while delta >= 1:
        print("{}-scaling phase".format(delta))
        ###print(pi)
        #TODO: it seems indeed as if we need to reset potentials in every delta scaling phase
        pi = {n:0 for n in G.nodes}
        R = compute_delta_residual_network(G,x,delta,pi)
        ###print_residual(R,G,x)
        ###print(x,e,delta)
        fulfill_reduced_cost_optimiality_rule(R,x,e,delta)
        ###print(x,e,delta)

        S = [n for n,k in e.items() if k>=delta]
        T = [n for n,k in e.items() if k<=-delta]
        
        while len(S) >0 and len(T)>0:
            k=S[-1]
            l=T[-1]
            r = get_unsaturated_residual_subgraph(R)
            distances, paths = nx.single_source_dijkstra(r,k,weight="rc")
            idx =-1
            while l not in paths:
                idx -=1
                l=T[idx]
            for n, d_value in distances.items():
                pi[n]-=d_value
            path = paths[l]
            
            #TODO: make sure order of the following two calls does in deed not matter
            R = recompute_reduced_cost(R,pi)
            augment_path(x,path,delta,R)
            
            print("augmented path:", path_str(path), "with:", delta)
            e[k]-=delta
            e[l]+=delta
            S = [n for n,k in e.items() if k>=delta]
            T = [n for n,k in e.items() if k<=-delta]
            print("|S(delta)|={} |T(delta)|={}".format(len(S),len(T)))
        delta = int(delta/2)
    return x
    
x = capacity_scaling_min_cost_flow("S","D",2,G)
print(x)
x = capacity_scaling_min_cost_flow(SRC,DEST,FLOW, channel_graph)

2-scaling phase
augmented path: S A B D with: 2
|S(delta)|=0 |T(delta)|=0
1-scaling phase
augmented path: S X Y D with: 1
|S(delta)|=0 |T(delta)|=0
{'S': {'A': 1, 'X': 1}, 'A': {'S': 0, 'B': 1}, 'X': {'S': 0, 'B': 0, 'Y': 1}, 'B': {'A': 0, 'X': 0, 'D': 1}, 'Y': {'X': 0, 'D': 1}, 'D': {'B': 0, 'Y': 0}}
8192-scaling phase
augmented path: 03efc 02247 0390b 022c6 with: 8192
|S(delta)|=0 |T(delta)|=0
4096-scaling phase
augmented path: 03efc 03cde 03864 022c6 with: 4096
|S(delta)|=0 |T(delta)|=0
2048-scaling phase
augmented path: 03efc 03267 022c6 with: 2048
|S(delta)|=1 |T(delta)|=1
augmented path: 03efc 02247 03d37 022c6 with: 2048
|S(delta)|=0 |T(delta)|=0
1024-scaling phase
augmented path: 02247 031ce 022c6 with: 1024
|S(delta)|=1 |T(delta)|=1
augmented path: 03efc 02247 022c2 022c6 with: 1024
|S(delta)|=1 |T(delta)|=1
augmented path: 03efc 03cde 03864 022c6 with: 1024
|S(delta)|=1 |T(delta)|=1
augmented path: 03efc 03267 022c6 with: 1024
|S(delta)|=0 |T(delta)|=0
512-scaling phase
augme

augmented path: 02a04 0288b 021c9 03d06 03e28 022c6 with: 4
|S(delta)|=3 |T(delta)|=3
augmented path: 02247 0390b 022c6 with: 4
|S(delta)|=3 |T(delta)|=3
augmented path: 02247 03d37 022c6 with: 4
|S(delta)|=3 |T(delta)|=3
augmented path: 02247 031ce 022c6 with: 4
|S(delta)|=2 |T(delta)|=3
augmented path: 03efc 02247 022c2 022c6 with: 4
|S(delta)|=2 |T(delta)|=3
augmented path: 03efc 03267 022c6 with: 4
|S(delta)|=2 |T(delta)|=3
augmented path: 03efc 03cde 030c3 02a04 02d4b 022c6 with: 4
|S(delta)|=2 |T(delta)|=3
augmented path: 03efc 033d8 0309b 022c6 with: 4
|S(delta)|=2 |T(delta)|=3
augmented path: 03efc 03abf 03864 039b1 022c6 with: 4
|S(delta)|=2 |T(delta)|=3
augmented path: 03efc 0213c 03cde 02004 033fd 022c6 with: 4
|S(delta)|=2 |T(delta)|=3
augmented path: 03efc 02537 03e29 03abf 035f5 022c6 with: 4
|S(delta)|=2 |T(delta)|=2
augmented path: 03efc 02b68 033d8 03d06 021c9 03864 with: 4
|S(delta)|=2 |T(delta)|=2
augmented path: 03efc 032cc 0260f 037f9 03864 with: 4
|S(delta)|=2 |T(

# Test if the result is indeed a min cost flow!
we have a minimum cost flow if the 1Residual net has no negative cycles!

In [220]:
pi = {n:0 for n in G.nodes}
R = compute_delta_residual_network(G,x,delta,pi)
if nx.negative_edge_cycle(G,weight="rc"):
    print("min cost flow not optimal")
else:
    print("min cost flow was optimal")


NameError: name 'delta' is not defined

In [218]:
print(SRC[:5],DEST[:5])
for src, v in x.items():
    for dest, f in v.items():
        if f>0:
            print(src[:5], dest[:5],f)

(cost(4096,9940)-cost(0,9940))/4096
print(2**-compute_cost(channel_graph,x))


03efc 022c6
03e28 022c6 512
03e28 033fd 11
03864 022c6 2048
03864 039b1 128
03864 021c9 409
03abf 03e28 1
03abf 03864 256
03abf 03fb8 1
03abf 035f5 31
03abf 02a04 256
03abf 028d9 15
03abf 037cc 16
03efc 03abf 512
03efc 02247 4096
03efc 03267 2048
03efc 033d8 256
03efc 03cde 2048
03efc 02b68 32
03efc 032cc 8
03efc 02160 8
03efc 02537 64
03efc 0213c 128
03baa 03864 65
02e90 03864 2
030c3 03baa 57
030c3 02a04 4
0260f 03864 8
0260f 03d06 2
0260f 037f9 6
0260f 02572 2
0260f 02d4b 2
0390b 022c6 1024
03d37 022c6 1024
03267 022c6 2048
03267 035f5 1
03fb8 022c6 1
02247 0390b 1024
02247 03d37 1024
02247 03267 1
02247 031ce 1024
02247 022c2 1023
031ce 022c6 1024
022c2 022c6 1024
02160 02e90 1
02160 0260f 6
02160 022c2 1
032cc 0260f 8
02004 033fd 4
03d06 03e28 478
03d06 03864 1
033fd 022c6 15
035f5 022c6 32
02a04 03864 128
02a04 02d4b 128
02a04 03d6b 1
02a04 0288b 3
039b1 022c6 128
02572 021c9 2
037f9 03864 6
02537 03e29 64
033d8 03864 63
033d8 0260f 2
033d8 03d06 4
033d8 02d4b 122
033d8 02dfd 1
0

In [96]:
def compute_cost(G,x):
    total_cost = 0
    for s,d in G.edges():
        c = G[s][d]["cap"]
        f = x[s][d]
        total_cost+=cost(f,c)
        f = x[d][s]
        total_cost+=cost(f,c)
        
    return total_cost

In [69]:
x = generate_null_flow(G)
print(compute_cost(G,x))
x["S"]["A"]=2
x["A"]["B"]=2
x["B"]["D"]=2
print(compute_cost(G,x))

x = generate_null_flow(G)
print(compute_cost(G,x))
x["S"]["A"]=1
x["A"]["B"]=1
x["B"]["D"]=1
x["S"]["X"]=1
x["X"]["Y"]=1
x["Y"]["D"]=1
print(compute_cost(G,x))



0.0
3.9068905956085183
0.0
3.0064262691594323


# OLD code approaches

In [53]:
import json
import networkx as nx
import math

log = math.log2

In [83]:
"""
def import_channel_graph():
    # retrieve this by: lightning-cli listchannels > listchannels.json
    f = open("listchannels.json")
    jsn = json.load(f)
    G = nx.Graph()
    for channel in jsn["channels"]:
        src = channel["source"]
        dest = channel["destination"]
        cap = int(int(channel["satoshis"])/1000)
        sid = channel["short_channel_id"]
        G.add_edge(src,dest,cap=cap,sid=sid)
    return G

channel_graph = import_channel_graph()

G = nx.DiGraph()
G.add_edge("S","A",cap=2)
G.add_edge("S","X",cap=1)
G.add_edge("A","B",cap=2)
G.add_edge("X","B",cap=9)
G.add_edge("X","Y",cap=7)
G.add_edge("Y","D",cap=4)
G.add_edge("B","D",cap=4)

def generate_null_flow(G):
    flow = {n:{} for n in G.nodes()}
    for u,v in G.edges():
        flow[u][v]=0
        flow[v][u]=0
    return flow

def next_hop(path):
    for i in range(1,len(path)):
        src = path[i-1]
        dest = path[i]
        yield (src,dest)
        
def handle_residual_edge(src,dest,R):

    if "orc" in R[src][dest]:
        R[src][dest]["rc"]=R[src][dest]["orc"]
        del R[src][dest]["orc"]
    else:
        R[src][dest]["orc"] = R[src][dest]["rc"]        
        del R[src][dest]["rc"] #this effectively removes edge from path computation

    if R.has_edge(dest,src):
        if  "rc" in R[dest][src]:
            R[dest][src]["orc"] = R[dest][src]["rc"]
            R[dest][src]["rc"] = 0
        else: 
            R.remove_edge(dest,src)
    else:
        R.add_edge(dest,src,rc=0)

        
        
    #R[src][dest]["orc"]=forward_rc

    
    
    #if R[src][dest]["rc"]==0 and "oldcost" in R[src][dest]:
    #    R[src][dest]["rc"] = R[src][dest]["oldcost"]
    


        
def augment_path(flow,path,amt,R):
    for src,dest in next_hop(path):
        if flow[dest][src]>0:
            flow[dest][src]-=amt
            #if flow
        else:
            flow[src][dest]+=amt
        handle_residual_edge(src,dest,R)

def cost(x,c):
    #if c%2==0:
        #print(x)
    #    return 4* x**2
    #else:
    #    return x**2
    return log(c+1)-log(c+1-x)

def path_str(path):
    it = path[0]
    if len(it)>5:
        it = it[:5]
    res = it
    for src, dest in next_hop(path):
        if len(dest)>5:
            dest = dest[:5]
        res += " {}".format(dest)
    return res

SRC = "03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df"
DEST = "022c699df736064b51a33017abfc4d577d133f7124ac117d3d9f9633b6297a3b6a"
FLOW = 9200

"""

In [84]:
def print_residual(r,G,x):
    for i,j in G.edges():
        cap = G[i][j]["cap"]
        f = x[i][j]
        print("src: {} dest: {} cap: {} flow: {}".format(i,j,cap,f))
        if r.has_edge(i,j):
            print(i,j,r[i][j]["weight"])
        if r.has_edge(j,i):
            print(j,i,r[j][i]["weight"])        

def compute_delta_residual_network(G,x,delta,C=cost):
    #####print("{}-Residual net for flow x={}".format(delta,x))
    residual = nx.DiGraph()
    for n in G.nodes():
        residual.add_node(n,demand=0)
    for i, j in G.edges():
        #print(i,j)
        cap = G[i][j]["cap"]
        f = x[i][j]
        
        if x[i][j] == 0 and x[j][i]==0: # no flow:             
            if delta<=cap:
                unit_cost = (C(f + delta, cap) - C(f,cap))/delta
                #print("add residual",i,j,unit_cost)
                
                residual.add_edge(i,j,weight=(C(f + delta, cap) - C(f,cap))/delta,capacity=delta)
                residual.add_edge(j,i,weight=(C(f + delta, cap) - C(f,cap))/delta,capacity=delta)
        if x[i][j] > 0 and x[j][i] > 0:
            print("FLOW IN BOTH DIRECTION CAN'T / SHOULDN'T HAPPEN",i,j, x[i][j],x[j][i])
        if x[i][j]>0:
            if f+delta <=cap:
                residual.add_edge(i,j,weight=(C(f + delta, cap) - C(f,cap))/delta,capacity=delta)
            if delta <= f:#backward flow
                #print("HERE!!!",C(f - delta, cap), - C(f,cap),delta)
                residual.add_edge(j,i,weight=(C(f - delta, cap) - C(f,cap))/(delta),capacity=delta)
        else: # if flow was in other direction enter in the other direction
            f=x[j][i]
            if f+delta <=cap:
                residual.add_edge(j,i,weight=(C(f + delta, cap) - C(f,cap))/delta,capacity=delta)
            if delta <= f:#backward flow
                residual.add_edge(i,j,weight=(C(f - delta, cap) - C(f,cap))/(delta),capacity=delta)
    #print_residual(residual,G,x)
    return residual
    
#compute_delta_residual_network(channel_graph,x,10)

In [85]:
def apply_reduced_cost(R,pi):
    for i,j in R.edges():
        R[i][j]["rc"]=R[i][j]["weight"]+pi[i]-pi[j]
        ###print(i,j,R[i][j]["rc"])

In [94]:
def fulfill_reduced_cost_optimiality_rule(R,pi,delta,e,x,G):
    #to_remove=[]
    for i,j in R.edges():
        #test and correct reduced cost optimality condition
        cap = 0
        if G.has_edge(i,j):
            cap = G[i][j]["cap"]
        if G.has_edge(j,i):
            cap = G[j][i]["cap"]
            
        if R[i][j]["weight"]+pi[i]-pi[j] <0: #TODO: also test R_ij capacity >= delta. so far only unit costs in residual net

            #might be too simplisic as flow in oposite direction might exist which we would have to decrease
            #####print("From {} to {} reduced cost: {} flow: {}".format(i,j,R[i][j]["weight"]+pi[i]-pi[j],x[j][i]),R[i][j]["weight"],pi[i],pi[j])

            #this means we save costs if we just ship from i to j!
            
            #edge can only have negative reduced cost in two cases: 
            # 1. there was flow in the other direction --> reduce by 1
            # 2. there was NO flow in either direction --> increase flow in oposite direction

            flag = False
            if x[j][i]>=delta:
                x[j][i]-=delta
                flag = True
                
            elif x[i][j]+delta <= cap:
                x[i][j]+=delta
                flag = True
                
            #if x[i][j]>0:
            #    print("!!!! ther was never flow in the direction in which the residual edges are not cost optimal",i,j,x[i][j])
            
            
            #TODO: test what happens if residual is recomputed
            if flag:
                R[i][j]["orc"]=0
                del R[i][j]["rc"]
                
                #to_remove.append((i,j))

                e[i]-=delta
                e[j]+=delta
    """        
    if len(to_remove)==0:
        return True
    for i,j in to_remove:
        R.remove_edge(i,j)
    #R = compute_delta_residual_network(G,x,delta)
    return False
    """

In [95]:
def has_negative_edges(R):
    for i,j in R.edges():
        if R[i][j]["rc"]<0:
            print("NEG EDGE!!!",R[i][j]["rc"],i,j)

In [96]:
w= open("out.text","w")
asdf = None
def cpacity_scaling(s,d,U,G):
    x = generate_null_flow(G)
    pi = {n:0 for n in G.nodes}
    e = {n:0 for n in G.nodes}
    e[s]=U
    e[d]=-U
    delta = 2**int(log(U))
    while delta >= 1:
        print("{}-scaling phase".format(delta))
        asdf = "asdf"#{k:v for k,v in x.items()}
        R = compute_delta_residual_network(G,x,delta)
        apply_reduced_cost(R,pi)
        #####print("excesses before optimality: ", e)
        #####print("flow before optimality", x)
        #####print("delta: ", delta)
        fulfill_reduced_cost_optimiality_rule(R,pi,delta,e,x,G)
        
        #####print("excesses: ", e)
        #####print("temporary flow", x) 
        #R = compute_delta_residual_network(G,x,delta)
        S = [n for n,k in e.items() if k>=delta]
        T = [n for n,k in e.items() if k<=-delta]
        #####print("S(delta): {}, T(delta):{}".format(S,T))
        
        
        ### APPROACH WITH SIMPLEX FLOW ON UNIT NETWORK
        #nx.set_node_attributes(R,"demand")
        """for k,v in e.items():
            if k in R.nodes() and v != 0:
                print(k,v)
                if v>delta:
                    R.nodes[k]["demand"]=delta
                elif v<-delta:
                    R.nodes[k]["demand"]=-delta
                else:
                    R.nodes[k]["demand"]=v
        
        print("go")
        res = nx.min_cost_flow(R)
        print("done",res)
        """

        while len(S) >0 and len(T)>0:
            k=S[-1]
            #l=T[-1]
            #S=S[:-1]
            #T=T[:-1]
            #path = nx.dijkstra_path(R,k,l)
            #apply_reduced_cost(R,pi)
            #has_negative_edges(R)
            distances, paths = nx.single_source_dijkstra(R,k,weight="rc")
            #distances, paths = nx.single_source_bellman_ford(R,k,weight="rc")
            #####print("Distances from {}".format(k))
            for n, d_value in distances.items():
                #####print(n,d_value)
                #if n in pi:
                pi[n]-=d_value
            
            ### TODO: we could actually not chose n but walk through the nodes in T until we find one that was reachable from k            
            #if l not in paths:
            #    w.write("has happend!")
            #    continue
            for l in T:
                if l in paths:
                    break
            
            path = paths[l]
            augment_path(x,path,delta,R)
            print("augmented path:", path_str(path), "with:", delta)
            e[k]-=delta
            e[l]+=delta
            ###print(e)
            #####print("flow: ",x)
            #R = compute_delta_residual_network(G,x,delta) # could be ment as remove edges of path instead of recompute
            #fulfill_reduced_cost_optimiality_rule(R,pi,delta,e,x,G)    
            """
            for src, dest in next_hop(path):
                #print(src, dest, x[src])
                if G.has_edge(src,dest):
                    ###TODO x[src][dest]+delta > G[src][dest]["cap"]
                    if x[src][dest]+delta>G[src][dest]["cap"]:
                        R.remove_edge(src,dest)
                if G.has_edge(dest,src):
                    if x[src][dest]+delta>G[dest][src]["cap"]:
                        R.remove_edge(src,dest)
            """

            #####print_residual(R,G,x)
            #####print("excesses: ",e)
            #apply_reduced_cost(R,pi)
            #if delta < 2:
            #fulfill_reduced_cost_optimiality_rule(R,pi,delta,e,x)
            S = [n for n,k in e.items() if k>=delta]
            T = [n for n,k in e.items() if k<=-delta]
            print("|S(delta)|={} |T(delta)|={}".format(len(S),len(T)))
        #pi = {k:v for k,v in PI.items()}
        delta = int(delta/2)
    return x
x = cpacity_scaling(SRC,DEST, FLOW, channel_graph)
#x = cpacity_scaling("S","D", 2, G)
print("min cost max flow: ", x)

8192-scaling phase
augmented path: 03efc 02247 0390b 022c6 with: 8192
|S(delta)|=0 |T(delta)|=0
4096-scaling phase
augmented path: 03d86 02616 03baa 030c3 02ad6 03f93 03fab with: 4096
|S(delta)|=127 |T(delta)|=675
augmented path: 0309a 021c9 03d06 0260f 02247 03fab with: 4096
|S(delta)|=126 |T(delta)|=675
augmented path: 02734 02816 03864 03fab with: 4096
|S(delta)|=125 |T(delta)|=674
augmented path: 03187 030c3 03cde 03864 03baa 02616 0260f 03d06 03e28 03b1b with: 4096
|S(delta)|=124 |T(delta)|=673
augmented path: 0231a 03376 02004 0309b 03e28 03d06 03baa 03434 with: 4096
|S(delta)|=124 |T(delta)|=672
augmented path: 0231a 0366d 03386 03e28 03d37 03b1b 03f93 033ac with: 4096
|S(delta)|=123 |T(delta)|=672
augmented path: 024ae 02717 0298f 03d06 0288b 03cde 030c3 02c91 03739 033ac with: 4096
|S(delta)|=122 |T(delta)|=671
augmented path: 039cb 03fab 03864 033d8 02da0 with: 4096
|S(delta)|=121 |T(delta)|=670
augmented path: 021c9 02572 035f5 02177 03d30 with: 4096
|S(delta)|=121 |T(delta)

augmented path: 03cde 03957 02c91 03583 with: 4096
|S(delta)|=91 |T(delta)|=622
augmented path: 03cde 03d28 03fb8 03a7f with: 4096
|S(delta)|=91 |T(delta)|=621
augmented path: 03cde 0242a 03267 03e5a 037a3 with: 4096
|S(delta)|=91 |T(delta)|=621
augmented path: 03cde 02b01 03e28 03267 037a3 with: 4096
|S(delta)|=91 |T(delta)|=620
augmented path: 03cde 02b68 028ed 02dfd 037f9 035f5 03613 with: 4096
|S(delta)|=91 |T(delta)|=619
augmented path: 03cde 03765 0390b 03267 0242a 03e24 with: 4096
|S(delta)|=91 |T(delta)|=618
augmented path: 03cde 03205 with: 4096
|S(delta)|=91 |T(delta)|=618
augmented path: 03cde 03c9b 03d6b 03e28 0309b 03205 with: 4096
|S(delta)|=91 |T(delta)|=618
augmented path: 03cde 02d3d 0215a 03205 with: 4096
|S(delta)|=91 |T(delta)|=618
augmented path: 03cde 0323f 0260f 033d8 03205 with: 4096
|S(delta)|=91 |T(delta)|=618
augmented path: 03cde 03eef 03bc9 02439 03d06 0213c 03da1 03205 with: 4096
|S(delta)|=91 |T(delta)|=617
augmented path: 03cde 02c19 0242a 03f02 with: 40

augmented path: 02a04 030c3 02fc8 031d2 with: 4096
|S(delta)|=64 |T(delta)|=551
augmented path: 02a04 03abf 03d37 0354f with: 4096
|S(delta)|=64 |T(delta)|=550
augmented path: 02a04 024d2 03950 02de6 03434 03f93 03b1b with: 4096
|S(delta)|=64 |T(delta)|=549
augmented path: 02a04 02b68 033d8 034cf with: 4096
|S(delta)|=64 |T(delta)|=548
augmented path: 02a04 031ce 0340c 02a5d with: 4096
|S(delta)|=64 |T(delta)|=547
augmented path: 02a04 02651 03e28 03add with: 4096
|S(delta)|=64 |T(delta)|=546
augmented path: 02a04 02c29 028a8 030f2 03267 0376a with: 4096
|S(delta)|=64 |T(delta)|=545
augmented path: 02a04 0303a 03186 with: 4096
|S(delta)|=63 |T(delta)|=545
augmented path: 02a5b 02275 0390b 03186 with: 4096
|S(delta)|=63 |T(delta)|=544
augmented path: 02a5b 028a8 02c91 03ec1 with: 4096
|S(delta)|=62 |T(delta)|=543
augmented path: 033e9 03864 03abf 03c2a 0260f 02160 032cc with: 4096
|S(delta)|=62 |T(delta)|=543
augmented path: 033e9 03fb8 0260f 032cc with: 4096
|S(delta)|=62 |T(delta)|=54

augmented path: 03d06 037f9 02b01 03abf 02c91 02cc4 with: 4096
|S(delta)|=56 |T(delta)|=467
augmented path: 03d06 02616 03267 02f0c with: 4096
|S(delta)|=56 |T(delta)|=466
augmented path: 03d06 03c2a 03e28 033fc 02275 03d40 with: 4096
|S(delta)|=56 |T(delta)|=465
augmented path: 03d06 038f8 03864 0284a 02c91 03d91 with: 4096
|S(delta)|=56 |T(delta)|=465
augmented path: 03d06 024f8 03bec 03267 03d91 with: 4096
|S(delta)|=56 |T(delta)|=464
augmented path: 03d06 03386 024d2 03267 035c7 with: 4096
|S(delta)|=56 |T(delta)|=463
augmented path: 03d06 02004 0260f 03abf 035f5 03902 with: 4096
|S(delta)|=56 |T(delta)|=462
augmented path: 03d06 02e8b 03bc9 02c91 02de6 with: 4096
|S(delta)|=56 |T(delta)|=461
augmented path: 03d06 03863 028a8 03e5e 02450 with: 4096
|S(delta)|=56 |T(delta)|=460
augmented path: 03d06 030a1 03baa 033e9 026be with: 4096
|S(delta)|=56 |T(delta)|=459
augmented path: 03d06 0298f 03180 02c91 024b7 with: 4096
|S(delta)|=56 |T(delta)|=458
augmented path: 03d06 02572 02de1 02

augmented path: 028a8 03abf 03037 02112 0242a 02779 with: 4096
|S(delta)|=40 |T(delta)|=407
augmented path: 028a8 02de1 02779 with: 4096
|S(delta)|=40 |T(delta)|=406
augmented path: 028a8 033d8 03375 03e28 021ec with: 4096
|S(delta)|=40 |T(delta)|=405
augmented path: 028a8 033e9 02234 with: 4096
|S(delta)|=40 |T(delta)|=404
augmented path: 028a8 03e28 02738 with: 4096
|S(delta)|=40 |T(delta)|=403
augmented path: 028a8 03267 03bec 03e28 02fe9 with: 4096
|S(delta)|=40 |T(delta)|=402
augmented path: 028a8 03863 with: 4096
|S(delta)|=39 |T(delta)|=401
augmented path: 03dab 02de1 03d97 with: 4096
|S(delta)|=39 |T(delta)|=400
augmented path: 03dab 02bb2 0388b with: 4096
|S(delta)|=39 |T(delta)|=399
augmented path: 03dab 03fb8 03bec 030c3 02c13 with: 4096
|S(delta)|=39 |T(delta)|=399
augmented path: 03dab 035f5 02de1 02c13 with: 4096
|S(delta)|=39 |T(delta)|=398
augmented path: 03dab 02572 03864 021c9 03d3c with: 4096
|S(delta)|=39 |T(delta)|=398
augmented path: 03dab 03baa 03d3c with: 4096
|

...

augmented path: 02e90 02ad6 030c3 02439 03da9 with: 2048
|S(delta)|=19 |T(delta)|=132
augmented path: 02e90 03bb8 023d7 03da9 with: 2048
|S(delta)|=19 |T(delta)|=132
augmented path: 02e90 03bb8 02a61 03269 02160 03944 02816 03da9 with: 2048
|S(delta)|=19 |T(delta)|=132
augmented path: 02e90 02731 032c6 02c91 0280b 0309b 034ea 03da9 with: 2048
|S(delta)|=19 |T(delta)|=132
augmented path: 02e90 02ad6 03da9 with: 2048
|S(delta)|=19 |T(delta)|=131
augmented path: 02e90 02ad6 030c3 0242a 02d45 03271 02e1d 024e0 with: 2048
|S(delta)|=19 |T(delta)|=131
augmented path: 02e90 03bb8 02d45 03271 02e1d 024e0 with: 2048
|S(delta)|=18 |T(delta)|=130
augmented path: 03baa 03bc9 030c3 02ad6 0293a with: 2048
|S(delta)|=18 |T(delta)|=130
augmented path: 03baa 03864 0293a with: 2048
|S(delta)|=18 |T(delta)|=129
augmented path: 03baa 03864 030c3 02e90 02c3f 0254a with: 2048
|S(delta)|=18 |T(delta)|=128
augmented path: 03baa 02b01 03269 02160 030c3 02e90 03bb8 02d45 0242a 02453 03691 with: 2048
|S(delta)|=

augmented path: 03abf 02f61 03295 0390b 03908 03634 03818 with: 2048
|S(delta)|=6 |T(delta)|=104
augmented path: 03abf 02265 02043 03dd1 03949 0327e 03634 03818 with: 2048
|S(delta)|=6 |T(delta)|=103
augmented path: 03abf 02e90 02ad6 02a19 with: 2048
|S(delta)|=6 |T(delta)|=103
augmented path: 03abf 02e90 02ad6 02a19 with: 2048
|S(delta)|=6 |T(delta)|=103
augmented path: 03abf 02342 02e90 02ad6 02a19 with: 2048
|S(delta)|=6 |T(delta)|=102
augmented path: 03abf 0360a 03cde 02d08 with: 2048
|S(delta)|=6 |T(delta)|=101
augmented path: 03abf 03bec 03d06 02d81 with: 2048
|S(delta)|=6 |T(delta)|=100
augmented path: 03abf 03bec 03d06 03450 with: 2048
|S(delta)|=6 |T(delta)|=99
augmented path: 03abf 03543 030c3 022b8 with: 2048
|S(delta)|=6 |T(delta)|=99
augmented path: 03abf 022b8 with: 2048
|S(delta)|=6 |T(delta)|=99
augmented path: 03abf 03554 0331f 022b8 with: 2048
|S(delta)|=6 |T(delta)|=99
augmented path: 03abf 0360a 03cde 02e96 03180 0209f 03864 022b8 with: 2048
|S(delta)|=6 |T(delta)|=

|S(delta)|=4 |T(delta)|=57
augmented path: 0242a 03fb8 0298f 030a1 with: 2048
|S(delta)|=4 |T(delta)|=56
augmented path: 0242a 02589 030c3 02a90 with: 2048
|S(delta)|=4 |T(delta)|=55
augmented path: 0242a 02589 030c3 03c2a 03abf 03c43 02b56 02529 02d41 036b9 with: 2048
|S(delta)|=4 |T(delta)|=55
augmented path: 0242a 03820 02d41 036b9 with: 2048
|S(delta)|=4 |T(delta)|=55
augmented path: 0242a 03820 02d41 036b9 with: 2048
|S(delta)|=4 |T(delta)|=54
augmented path: 0242a 03a9b 02160 031d2 02fc8 03bb8 03011 with: 2048
|S(delta)|=4 |T(delta)|=54
augmented path: 0242a 03011 with: 2048
|S(delta)|=4 |T(delta)|=54
augmented path: 0242a 03011 with: 2048
|S(delta)|=4 |T(delta)|=54
augmented path: 0242a 03011 with: 2048
|S(delta)|=4 |T(delta)|=54
augmented path: 0242a 03011 with: 2048
|S(delta)|=4 |T(delta)|=54
augmented path: 0242a 03011 with: 2048
|S(delta)|=4 |T(delta)|=54
augmented path: 0242a 03011 with: 2048
|S(delta)|=4 |T(delta)|=54
augmented path: 0242a 03011 with: 2048
|S(delta)|=4 |T(

augmented path: 03864 02734 with: 2048
|S(delta)|=3 |T(delta)|=17
augmented path: 03864 03258 02a44 02439 030c3 03735 with: 2048
|S(delta)|=3 |T(delta)|=16
augmented path: 03864 0347b with: 2048
|S(delta)|=3 |T(delta)|=16
augmented path: 03864 0347b with: 2048
|S(delta)|=3 |T(delta)|=15
augmented path: 03864 03258 02b4a 0202e 033d8 03e5a 039ed 03bb8 021f1 with: 2048
|S(delta)|=3 |T(delta)|=15
augmented path: 03864 03bb8 021f1 with: 2048
|S(delta)|=3 |T(delta)|=14
augmented path: 03864 03bb8 02459 030c3 03da6 with: 2048
|S(delta)|=3 |T(delta)|=13
augmented path: 03864 03112 with: 2048
|S(delta)|=3 |T(delta)|=12
augmented path: 03864 0340f 03cde 03f36 03d06 0204a with: 2048
|S(delta)|=3 |T(delta)|=12
augmented path: 03864 0242a 03f36 03d06 0204a with: 2048
|S(delta)|=3 |T(delta)|=12
augmented path: 03864 02734 02d3d 02e7c 0204a with: 2048
|S(delta)|=3 |T(delta)|=11
augmented path: 03864 03765 03332 with: 2048
|S(delta)|=3 |T(delta)|=11
augmented path: 03864 03765 03332 with: 2048
|S(delt

ValueError: math domain error

In [ ]:
for i,j in channel_graph.edges():
    cap = channel_graph[i][j]["cap"]
    if cap < g[i][j]:
        print(i,j,cap,g[i][j])
    if cap < g[j][i]:
        print(j,i,cap,g[j][i])
    if g[i][j] < 0:
        print(i,j,cap,g[i][j])
    if g[j][i] < 0:
        print(j,i,cap,g[j][i])
    
        

In [47]:
#print("from {} to {}".format(SRC[:5],DEST[:5]))
for s,v in x.items():
    for d,amt in v.items():
        if amt > 0:
            print(s,d,amt, channel_graph[s][d]["cap"])

03864ef025fde8fb587d989186ce6a4a186895ee44a926bfc370e2c366597a3f8f 022c699df736064b51a33017abfc4d577d133f7124ac117d3d9f9633b6297a3b6a 1 15000
03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df 02247d9db0dfafea745ef8c9e161eb322f73ac3f8858d8730b6fd97254747ce76b 1025 16777
03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df 032679fec1213e5b0a23e066c019d7b991b95c6e4d28806b9ebd1362f9e32775cf 1024 9000
03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df 03cde60a6323f7122d5178255766e38114b4722ede08f7c9e0c5df9b912cc201d6 1 10000
032679fec1213e5b0a23e066c019d7b991b95c6e4d28806b9ebd1362f9e32775cf 022c699df736064b51a33017abfc4d577d133f7124ac117d3d9f9633b6297a3b6a 2049 16777
02247d9db0dfafea745ef8c9e161eb322f73ac3f8858d8730b6fd97254747ce76b 032679fec1213e5b0a23e066c019d7b991b95c6e4d28806b9ebd1362f9e32775cf 1025 16777
03cde60a6323f7122d5178255766e38114b4722ede08f7c9e0c5df9b912cc201d6 03864ef025fde8fb587d989186ce6a4a186895ee44a926bfc370e2c366597a3f8f 1 2

In [90]:
asdf